In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from scipy import constants as sc
from DeepFMKit.plotting import default_rc

plt.rcParams.update(default_rc)

def plot_ambiguity_map_panel(ax, dl, m_true, sigma_m_range, sigma_df_frac_range):
    """
    Generates a single panel of the ambiguity resolution heatmap for a given
    set of system parameters.
    """
    # --- 1. Define System Constants for this panel ---
    df_true_Hz = m_true * sc.c / (2 * np.pi * dl)
    lambda0_m = 1064e-9 # Switched to 1064nm for consistency with paper
    f0_Hz = sc.c / lambda0_m
    
    # --- 2. Define the Uncertainty Axes ---
    sigma_m_axis = np.linspace(0, sigma_m_range, 50)
    sigma_df_frac_axis = np.linspace(0, sigma_df_frac_range, 50)

    sigma_m_grid, sigma_df_frac_grid = np.meshgrid(sigma_m_axis, sigma_df_frac_axis)
    sigma_df_grid = sigma_df_frac_grid * df_true_Hz

    # --- 3. Calculate the Total Uncertainty in the Coarse Phase ---
    term1_var = (-f0_Hz / df_true_Hz)**2 * sigma_m_grid**2
    term2_var = (m_true * f0_Hz / df_true_Hz**2)**2 * sigma_df_grid**2
    sigma_phi_coarse_grid = np.sqrt(term1_var + term2_var)

    # --- 4. Create the Heatmap Plot ---
    cmap = 'YlGnBu'
    norm = mcolors.TwoSlopeNorm(vcenter=np.pi, vmin=0, vmax=2*np.pi)

    pcm = ax.pcolormesh(
        sigma_m_axis * 1e6, # Plot axis in ppm
        sigma_df_frac_axis * 1e6, # Plot axis in ppm
        sigma_phi_coarse_grid,
        cmap=cmap,
        norm=norm,
        shading='gouraud'
    )

    # --- 5. Add the Critical Contour Line ---
    ax.contour(
        sigma_m_axis * 1e6,
        sigma_df_frac_axis * 1e6,
        sigma_phi_coarse_grid,
        levels=[np.pi],
        colors='black',
        linewidths=1.5,
        linestyles='dashed'
    )
    
    # --- 6. Formatting ---
    ax.set_xlabel(r'Uncertainty in $\hat{m}$, $\sigma_m$ ($\mu$rad)')
    ax.set_ylabel(r'Uncertainty in $\Delta f_{\\rm est}}$, $\sigma_{\Delta f}/\Delta f$ (ppm)')
    ax.set_title(r'$\Delta l=$' + f'{dl*100:.0f}' +r'$\,{\rm cm}, m=$' + f'{m_true:.1f}' + r', $\Delta f=$' + f'{df_true_Hz/1e9:.1f}' + r'$\,{\rm GHz}$')
    ax.grid(False)
    
    return pcm

# --- Create the two-panel figure ---
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(6.875, 3.0), dpi=150, sharey=False)

# Panel 1: Short OPD, low m
pcm1 = plot_ambiguity_map_panel(ax1, dl=0.1, m_true=2*np.pi, sigma_m_range=100e-6, sigma_df_frac_range=10e-6)
ax1.set_ylabel(r"Uncertainty in $\Delta f_{\rm est}$, $\sigma_{\Delta f}/\Delta f$ (ppm)")


# Panel 2: Longer OPD, high m
pcm2 = plot_ambiguity_map_panel(ax2, dl=0.1, m_true=4*np.pi, sigma_m_range=100e-6, sigma_df_frac_range=10e-6)
ax2.set_ylabel('') # Hide y-label for the second panel to avoid clutter

# Add a single, shared colorbar
fig.subplots_adjust(right=0.85, wspace=0.2)
cbar_ax = fig.add_axes([0.87, 0.12, 0.015, 0.75])
cbar = fig.colorbar(pcm1, cax=cbar_ax)
cbar.set_label(r'Coarse phase uncertainty, $\sigma_{\Phi,\rm coarse}$ (rad)')

plt.show()
fig.savefig("ambiguity_resolution_10cm.pdf", bbox_inches='tight')